In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Masking
from tensorflow.keras.optimizers.legacy import RMSprop

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
import pickle as pkl
from src.utils.data_transform import *
import pandas as pd
import os 
import pickle 
import json
from src.utils.data_io import *

In [ ]:
def build_model():    
    model = Sequential([
        Masking(mask_value=0., input_shape=(90, 5)),  # Masking layer to ignore padded zeros
        LSTM(64, activation='tanh', recurrent_activation='hard_sigmoid', return_sequences=True),
        LSTM(64, activation='tanh', recurrent_activation='hard_sigmoid'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')
    ])
    
    # Compile the model
    model.compile(optimizer=RMSprop(learning_rate=1e-3), loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
def build_model_2():
    model = Sequential([
        Masking(mask_value=0., input_shape=(90, 5)),  # Masking layer to ignore padded zeros
        LSTM(64, activation='tanh', recurrent_activation='sigmoid', use_bias=True, recurrent_dropout=0, unroll=False, return_sequences=True),
        LSTM(64, activation='tanh', recurrent_activation='sigmoid', use_bias=True, recurrent_dropout=0, unroll=False),
        Dropout(0.5),
        Dense(1, activation='sigmoid')
    ])
    
    # Compile the model
    model.compile(optimizer=RMSprop(learning_rate=1e-3), loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
with open("../../data/dataset-info-json/subject_to_indices.json", "r") as f:
    subject_to_indices = json.load(f)

subject_to_indices = {int(k): v for k, v in subject_to_indices.items()}

In [ ]:
with open("../../data/lstm_training_data/processed/balanced_training_data.pkl", "rb") as f:
    data = pickle.load(f)

**Train in non academic loso**

In [ ]:
for test_subject in subject_to_indices.keys():
    print(f"Processing test subject {test_subject}...")
    results = []
    accuracy = []
    loss = []
    # Load data for the current LOSO split
    training_data, training_labels, testing_data, testing_labels = load_data(test_subject, subject_to_indices, data)
    model = build_model_2()
    history = model.fit(training_data, training_labels, epochs=6, batch_size=32)
    results.append(model.evaluate(testing_data, testing_labels))
    accuracy.append(history.history['accuracy'])
    loss.append(history.history['loss'])
    
    model.save(f"../../models/lstm/EFF_model_{test_subject}.keras")
    
    training_info_path = "../../models/lstm/training_info/"
    os.makedirs(training_info_path, exist_ok=True)
    save_data(results, training_info_path, f"EFFresults_{test_subject}")
    save_data(accuracy, training_info_path, f"EFFaccuracy_{test_subject}")
    save_data(loss, training_info_path, f"EFFloss_{test_subject}")

**Tuning with callbacks**

In [ ]:
import os
import numpy as np
import datetime
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint
from tensorflow.keras.models import load_model

subject = 1
training_data, training_labels, testing_data, testing_labels = load_prediction_data(subject, subject_to_indices, data)
full_data = np.concatenate((training_data, testing_data), axis=0)
full_labels = np.concatenate((training_labels, testing_labels), axis=0)

model_tune = build_model_2()

# Callbacks setup
path = "../../models/lstm/tuning/100/"
log_dir = os.path.join(path, "logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
checkpoint_dir = os.path.join(path, "checkpoints")
os.makedirs(log_dir, exist_ok=True)
os.makedirs(checkpoint_dir, exist_ok=True)
checkpoint_path = os.path.join(checkpoint_dir, "cp-{epoch:04d}.ckpt")

tensorboard_callback = TensorBoard(
    log_dir=log_dir,
    histogram_freq=1,
    write_graph=True,
    write_images=True,
    update_freq='epoch',
    profile_batch=2,
    embeddings_freq=1,
)

model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    save_weights_only=True,
    verbose=1,
    save_freq='epoch',
    monitor='accuracy',
    save_best_only=True,
    mode='max'
)

# Model training with callbacks
history_tune = model_tune.fit(
    full_data,
    full_labels,
    epochs=100,
    batch_size=32,
    callbacks=[tensorboard_callback, model_checkpoint_callback]
)

model_tune.save(f"{path}/model_100.keras")

accuracy_tune = history_tune.history["accuracy"]
loss_tune = history_tune.history["loss"]

save_data(accuracy_tune, f"{path}/training_info", "accuracy_100")
save_data(loss_tune, f"{path}/training_info",  "loss_100")

**Load checkpoint weights and save model**

In [ ]:
checkpoint_dir = "../../models/lstm/tuning/100/checkpoints/cp-0093.ckpt"
ckpt_model = build_model_2()
ckpt_model.load_weights(checkpoint_dir)
ckpt_model.save(f"{path}/model_93.keras")

In [ ]:
%matplotlib notebook
path = "../../models/lstm/tuning/100/"

plot_metric(accuracy_tune, "Accuracy", f"{path}figs/training_accuracy.svg")
plot_metric(loss_tune, "Loss", f"{path}figs/training_loss.svg")
